### Info
based on notebook4

compare qkeras and qonnx predictions (without using hls4ml)\
strange thing on github: pred_qonnx[0]: [[4.4136963]]; then: 4.4375

### Load packages and the model

In [1]:
import tensorflow as tf
import numpy as np
import h5py
from qkeras.utils import _add_supported_quantized_objects
from qkeras import quantized_bits
co = {}; _add_supported_quantized_objects(co)

model_path = f"/eos/project/c/cms-l1ml/public/Pawel/axol1tl_model.h5"

model = tf.keras.models.load_model(model_path, custom_objects=co)
#model_config = model.get_config()

data_path = f"/eos/project/c/cms-l1ml/public/Pawel/complete.h5"
data_file = h5py.File(data_path, 'r')

2025-09-25 12:42:40.395909: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-25 12:42:41.049541: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-09-25 12:42:41.049587: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-09-25 12:42:41.053486: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-25 12:42:41.459049: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-25 12:42:41.462701: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

### Copy weights

In [3]:
ds_weights = data_file['model']['trimmed_encoder']['model_weights']
for layer in iter(model.layers):
    #print(layer.name)
    if layer.name in ('input_1','subtract','dot'): continue

    if layer.name == 'q_dense_4':
        weights_group = ds_weights['q_dense_4']['variational_auto_encoder']['vae__encoder']['q_dense_4']
        layer.set_weights((weights_group['kernel:0'], weights_group['bias:0']))
        continue

    weights_group = ds_weights[layer.name][layer.name]
    layer.set_weights((weights_group['kernel:0'], weights_group['bias:0']))

### QONNX model

In [4]:
import qonnx.core.onnx_exec as oxe
from qonnx.converters import from_keras
from qonnx.core.modelwrapper import ModelWrapper
#from qonnx.transformation.infer_shapes import InferShapes

qonnx_model, external_storage = from_keras(model)
qonnx_model = ModelWrapper(qonnx_model)

/afs/cern.ch/user/p/psajdak/eos_mine/project_CERN/venv3_9/lib64/python3.9/site-packages/keras/src/constraints.py:365: UserWarning: The `keras.constraints.serialize()` API should only be used for objects of type `keras.constraints.Constraint`. Found an instance of type <class 'qkeras.quantizers.quantized_bits'>, which may lead to improper serialization.
  warnings.warn(
2025-09-25 12:45:53.678563: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2025-09-25 12:45:53.678965: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2025-09-25 12:45:53.843691: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2025-09-25 12:45:53.843889: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session


In [89]:
for node in qonnx_model.graph.node:
    print(node.name)

Quant_0
Quant_1
Quant_2
Quant_3
Quant_4
Quant_5
Quant_6
Quant_7
Quant_8
Quant_9
Quant_10
Quant_11
Quant_12
Quant_13
Quant_14
Quant_15
MatMul_0
Add_0
Quant_16
Relu_0
Quant_17
MatMul_1
Add_1
Quant_18
Relu_1
Quant_19
MatMul_2
Add_2
Quant_20
Relu_2
Quant_21
MatMul_3
Add_3
Quant_22
Relu_3
Quant_23
MatMul_4
Add_4
MatMul_5
Add_5
Quant_24
Relu_4
Quant_25
MatMul_6
Add_6
Quant_26
Relu_5
Quant_27
MatMul_7
Add_7
Sub_0
Unsqueeze_0
Unsqueeze_1
MatMul_8
Squeeze_0
Identity_0
Identity_1


In [ ]:
quant_bit_widths = []
for node in qonnx_model.get_finn_nodes():
    quant_bit_widths.append(qonnx_model.get_initializer(node.input[3]))

print(quant_bit_widths)

[array(6), array(6), array(6), array(6), array(6), array(6), array(6), array(10), array(10), array(10), array(10), array(10), array(10), array(10), array(6), array(10), array(10), array(10), array(10), array(10), array(10), array(10), array(10), array(10), array(10), array(10), array(10), array(10)]


Quant node is not present between each add and relu within the Dense layers.\
These quant nodes should correspond to accumulators (?)